To start, pandas will be used to read the csv file of the dataset, so that it can be later cleaned with pandas.

In [50]:
import pandas as pd

air = pd.read_csv("Air_Quality_Dataset.csv")

print(air.head())
print(air.dtypes)
print(air.nunique())

   MeasureId                                        MeasureName MeasureType  \
0         83  Number of days with maximum 8-hour average ozo...      Counts   
1         83  Number of days with maximum 8-hour average ozo...      Counts   
2         83  Number of days with maximum 8-hour average ozo...      Counts   
3         83  Number of days with maximum 8-hour average ozo...      Counts   
4         83  Number of days with maximum 8-hour average ozo...      Counts   

  StratificationLevel  StateFips StateName  CountyFips CountyName  ReportYear  \
0      State x County          1   Alabama        1027       Clay        1999   
1      State x County          1   Alabama        1051     Elmore        1999   
2      State x County          1   Alabama        1073  Jefferson        1999   
3      State x County          1   Alabama        1079   Lawrence        1999   
4      State x County          1   Alabama        1089    Madison        1999   

   Value      Unit  UnitName    DataOr

The two most important things from this general overview are that most of the datatypes are objects, which will eventually need to be changed. Also, the single-digit StateFips and CountyFips are not adequately padded and therefore cannot be recognized as valid FIPS codes.

# Cleaning and standardizing the fips columns

This will allow for easier comparisons and merging with the walkability dataset.

In [53]:
# pads county FIPS to fit within the 5-digit rule
air["CountyFips"] = air["CountyFips"].astype(str).str.zfill(5)

# pads state fips within the 2-digit rule
air["StateFips"]= air["StateFips"].astype(str).str.zfill(2)

# create a unique column for just a county FIP code
air["COUNTYFP"] = air["CountyFips"].astype(str).str[-3:]

# test to make sure the process above was handled correctly
print(air.head())
print(air.dtypes)

   MeasureId                                        MeasureName MeasureType  \
0         83  Number of days with maximum 8-hour average ozo...      Counts   
1         83  Number of days with maximum 8-hour average ozo...      Counts   
2         83  Number of days with maximum 8-hour average ozo...      Counts   
3         83  Number of days with maximum 8-hour average ozo...      Counts   
4         83  Number of days with maximum 8-hour average ozo...      Counts   

  StratificationLevel StateFips StateName CountyFips CountyName  ReportYear  \
0      State x County        01   Alabama      01027       Clay        1999   
1      State x County        01   Alabama      01051     Elmore        1999   
2      State x County        01   Alabama      01073  Jefferson        1999   
3      State x County        01   Alabama      01079   Lawrence        1999   
4      State x County        01   Alabama      01089    Madison        1999   

   Value      Unit  UnitName    DataOrigin  Monito

# Checking for missing data

In [55]:
print(air.isnull().sum())

MeasureId              0
MeasureName            0
MeasureType            0
StratificationLevel    0
StateFips              0
StateName              0
CountyFips             0
CountyName             0
ReportYear             0
Value                  0
Unit                   0
UnitName               0
DataOrigin             0
MonitorOnly            0
COUNTYFP               0
dtype: int64


Wow! No missing data!

# Standardize column types

Creates consistent data types for more seamless data manipulation and comparison with other dataset.

In [59]:
# changing FIP columns to int64
air["COUNTYFP"] = air["COUNTYFP"].astype('int64')
air["CountyFips"] = air["CountyFips"].astype('int64')
air["StateFips"] = air["StateFips"].astype('int64')

# changing name columns to strings
air["CountyName"] = air["CountyName"].astype('string')
air["StateName"] = air["StateName"].astype('string')
air["MeasureName"] = air["MeasureName"].astype('string')
air["UnitName"] = air["UnitName"].astype('string')

# test to see changes
print(air.dtypes)

MeasureId                       int64
MeasureName            string[python]
MeasureType                    object
StratificationLevel            object
StateFips                       int64
StateName              string[python]
CountyFips                      int64
CountyName             string[python]
ReportYear                      int64
Value                         float64
Unit                           object
UnitName               string[python]
DataOrigin                     object
MonitorOnly                     int64
COUNTYFP                        int64
dtype: object


# What areas are included in the dataset?

This will help us determine whether one dataset has any areas that the other doesn't.

In [62]:
print(air["StateName"].unique())

<StringArray>
[             'Alabama',               'Alaska',              'Arizona',
             'Arkansas',           'California',             'Oklahoma',
             'Colorado',                 'Ohio',          'Connecticut',
             'Delaware', 'District of Columbia',              'Florida',
              'Georgia',               'Hawaii',                'Idaho',
             'Illinois',              'Indiana',                 'Iowa',
               'Kansas',             'Kentucky',            'Louisiana',
                'Maine',             'Maryland',        'Massachusetts',
             'Michigan',            'Minnesota',          'Mississippi',
             'Missouri',              'Montana',             'Nebraska',
               'Nevada',           'New Mexico',             'New York',
       'North Carolina',                'Texas',         'North Dakota',
               'Oregon',         'Pennsylvania',       'South Carolina',
         'South Dakota',            '

Turns out that this data set only uses states (and DC) with no outside territories, which the other dataset does include.

This tells us the period of time this dataset covers and shows that the data were collected over about 10 and a half years. Which, for what we are analysing, is a perfect amount of time.

In [63]:
print(air["ReportYear"].unique())

[1999 2013 2006 2002 2003 2011 2007 2008 2004 2009 2012 2010 2000 2001
 2005]


# What methods were used to collect the data?

Finding the unique measure names will tell us which methods were used to collect the data.

In [67]:
method = air["MeasureName"].unique()

for i in method:
    print(i)

Number of days with maximum 8-hour average ozone concentration over the National Ambient Air Quality Standard
Percent of days with PM2.5 levels over the National Ambient Air Quality Standard (NAAQS)
Person-days with PM2.5 over the National Ambient Air Quality Standard
Number of person-days with maximum 8-hour average ozone concentration over the National Ambient Air Quality Standard
Annual average ambient concentrations of PM2.5 in micrograms per cubic meter (based on seasonal averages and daily measurement)
Number of days with maximum 8-hour average ozone concentration over the National Ambient Air Quality Standard (monitor and modeled data)
Number of person-days with maximum 8-hour average ozone concentration over the National Ambient Air Quality Standard (monitor and modeled data)
Percent of days with PM2.5 levels over the National Ambient Air Quality Standard (monitor and modeled data)
Number of person-days with PM2.5 over the National Ambient Air Quality Standard (monitor and mode

Many of these methods are interesting and can yield unique analyses. However, due to the sheer size and specificity of the other dataset, we wanted to choose a method that would simplify the data and be best suited for comparison with walkability. This landed us on the "Annual average ambient concentrations of PM2.5 in micrograms per cubic meter."

# Filtering the PM2.5 “annual average” measure

This is the standard often used by the EPA and other organizations. Where 0-9 µg/m³ is considered safe enough, and anything above that is unhealthy. (Sidenote: There is actually no safe amount)

In [70]:
pm25 = air[
    (air["MeasureType"] == "Average") & (air["Unit"] == "µg/m³")] # yearly averages in micrograms per cubic meter

pm25.head()

,MeasureId,MeasureName,MeasureType,StratificationLevel,StateFips,StateName,CountyFips,CountyName,ReportYear,Value,Unit,UnitName,DataOrigin,MonitorOnly,COUNTYFP
39218,87,Annual average ambient concentrations of PM2.5...,Average,State x County,54,West Virginia,54081,Raleigh,2003,12.349080,µg/m³,Micograms per cubic meter,Monitor Only,1,81
39219,87,Annual average ambient concentrations of PM2.5...,Average,State x County,12,Florida,12033,Escambia,2002,10.953455,µg/m³,Micograms per cubic meter,Monitor Only,1,33
39220,87,Annual average ambient concentrations of PM2.5...,Average,State x County,29,Missouri,29099,Jefferson,1999,15.155819,µg/m³,Micograms per cubic meter,Monitor Only,1,99
39221,87,Annual average ambient concentrations of PM2.5...,Average,State x County,30,Montana,30029,Flathead,2001,8.272648,µg/m³,Micograms per cubic meter,Monitor Only,1,29
39222,87,Annual average ambient concentrations of PM2.5...,Average,State x County,12,Florida,12103,Pinellas,2006,9.553561,µg/m³,Micograms per cubic meter,Monitor Only,1,103


# Computing the 15 year average for PM2.5 measure

In [73]:
avg_pm25 = (pm25.groupby(["StateFips", "CountyFips", "StateName", "CountyName"], as_index=False)["Value"].mean())

avg_pm25.rename(columns={"Value": "PM25_15yr_Avg"}, inplace=True)

avg_pm25.head()

,StateFips,CountyFips,StateName,CountyName,PM25_15yr_Avg
0,1,1001,Alabama,Autauga,12.579127
1,1,1003,Alabama,Baldwin,10.861921
2,1,1005,Alabama,Barbour,11.998740
3,1,1007,Alabama,Bibb,12.715060
4,1,1009,Alabama,Blount,14.106742


In [75]:
avg_pm25[avg_pm25['CountyFips'] == 25027]

,StateFips,CountyFips,StateName,CountyName,PM25_15yr_Avg
1204,25,25027,Massachusetts,Worcester,10.877941


# Padding FIP codes

Fixing some of the 4-digit FIP codes that were accidentally made.

In [78]:
# initial check to show a 4-number FIP code is in the data.
avg_pm25[avg_pm25['CountyFips'] == 9015]

,StateFips,CountyFips,StateName,CountyName,PM25_15yr_Avg
292,9,9015,Connecticut,Windham,10.598999


In [80]:
# changing the FIP codes into strings for easier padding and consistent data types
avg_pm25["StateFips"] = avg_pm25["StateFips"].astype("string")
avg_pm25["CountyFips"] = avg_pm25["CountyFips"].astype("string")
avg_pm25.dtypes

StateFips        string[python]
CountyFips       string[python]
StateName        string[python]
CountyName       string[python]
PM25_15yr_Avg           float64
dtype: object

In [81]:
avg_pm25["CountyFips"] = avg_pm25["CountyFips"].str.zfill(5)
avg_pm25["StateFips"] = avg_pm25["StateFips"].str.zfill(2)

In [84]:
# check to see if cleaning worked
avg_pm25[avg_pm25['CountyFips'] == "09015"]

,StateFips,CountyFips,StateName,CountyName,PM25_15yr_Avg
292,09,09015,Connecticut,Windham,10.598999


# Changing names to match other dataset for easier joinging

In [86]:
avg_pm25 = avg_pm25.rename(columns = {'CountyFips' : 'STC_Fips', 'StateFips' : 'STATEFP'})

avg_pm25.head()

,STATEFP,STC_Fips,StateName,CountyName,PM25_15yr_Avg
0,01,01001,Alabama,Autauga,12.579127
1,01,01003,Alabama,Baldwin,10.861921
2,01,01005,Alabama,Barbour,11.998740
3,01,01007,Alabama,Bibb,12.715060
4,01,01009,Alabama,Blount,14.106742


In [89]:
avg_pm25.set_index(["STATEFP","STC_Fips"],inplace=True)

display(avg_pm25)

StateName  CountyName  PM25_15yr_Avg
STATEFP STC_Fips                                     
01      01001      Alabama     Autauga      12.579127
        01003      Alabama     Baldwin      10.861921
        01005      Alabama     Barbour      11.998740
        01007      Alabama        Bibb      12.715060
        01009      Alabama      Blount      14.106742
...                    ...         ...            ...
56      56037      Wyoming  Sweetwater       7.483988
        56039      Wyoming       Teton       6.342747
        56041      Wyoming       Uinta       8.197725
        56043      Wyoming    Washakie       6.614564
        56045      Wyoming      Weston       6.494029

[3117 rows x 3 columns]

# Export

This will then be exported into a csv that can be uploaded to our database through MongoCompass.

In [91]:
# This is directly imported through mongocompass
avg_pm25.to_csv('avg_pm25.csv')